# RTF extraction

This notebook shows examples of text extraction from RTF files with different packages

**Table of contents**<a id='toc0_'></a>    
- 1. [Methods to load RTF files](#toc1_)    
  - 1.1. [Load from unstructured local RTF loader](#toc1_1_)    
  - 1.2. [Load from unstructured io API](#toc1_2_)    
- 2. [Evaluate loded docs by embedding similarity](#toc2_)    
  - 2.1. [Embedding & Storage](#toc2_1_)    
  - 2.2. [Similarity search](#toc2_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

import glob
import pandas as pd
from dotenv import load_dotenv
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from tqdm.autonotebook import trange


## 1. <a id='toc1_'></a>[Methods to load RTF files](#toc0_)

In [6]:
folder_loc = os.path.join(kit_dir,'data/sample_data/sample_files/')
rtf_files = list(glob.glob(f'{folder_loc}/*.rtf'))
file_path = rtf_files[0]


##### Load text splitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        # Set a small chunk size, just to make splitting evident.
        chunk_size = 200,
        chunk_overlap  = 20,
        length_function = len,
        add_start_index = True,
        separators = ["\n\n\n","\n\n", "\n", "."]
    )

### 1.1. <a id='toc1_1_'></a>[Load from unstructured local RTF loader](#toc0_)

for using pypandoc is it required to install pandoc -> https://pandoc.org/installing.html

In [ ]:
from langchain_classic.document_loaders import UnstructuredRTFLoader

loader = UnstructuredRTFLoader(file_path, mode="elements")
docs_unstructured_local = loader.load_and_split(text_splitter = text_splitter)
for doc in docs_unstructured_local:
    print(f'{doc.page_content}\n---')

My First Heading
---
My first paragraph.
---
Table Example:
---
Column 1 Column 2 Row 1, Cell 1 Row 1, Cell 2 Row 2, Cell 1 Row 2, Cell 2
---


### 1.2. <a id='toc1_2_'></a>[Load from unstructured io API](#toc0_)

In [ ]:
from langchain_classic.document_loaders import UnstructuredAPIFileLoader
# register at Unstructured.io to get a free API Key
load_dotenv(os.path.join(repo_dir,'.env'))

loader = UnstructuredAPIFileLoader(file_path, 
                                   mode="elements", 
                                   api_key=os.environ.get('UNSTRUCTURED_API_KEY'),
                                   url=os.environ.get("UNSTRUCTURED_URL"))
docs_unstructured_api = loader.load_and_split(text_splitter = text_splitter)
for doc in docs_unstructured_api:
    print(f'{doc.page_content}\n---')

My First Heading
---
My first paragraph.
---
Table Example:
---
Column 1 Column 2 Row 1, Cell 1 Row 1, Cell 2 Row 2, Cell 1 Row 2, Cell 2
---


## 2. <a id='toc2_'></a>[Evaluate loded docs by embedding similarity](#toc0_)

### 2.1. <a id='toc2_1_'></a>[Embedding & Storage](#toc0_)

In [ ]:
from langchain_classic.embeddings import HuggingFaceInstructEmbeddings
from langchain_classic.vectorstores import FAISS

encode_kwargs = {'normalize_embeddings': True}
embd_model = HuggingFaceInstructEmbeddings( model_name='intfloat/e5-large-v2',
                                            embed_instruction="", # no instructions needed for candidate passages
                                            query_instruction="Represent this sentence for searching relevant passages: ",
                                            encode_kwargs=encode_kwargs)
vectorstore_unstructured_local = FAISS.from_documents(documents=docs_unstructured_local, embedding=embd_model)
vectorstore_unstructured_api = FAISS.from_documents(documents=docs_unstructured_api, embedding=embd_model)

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.2. <a id='toc2_2_'></a>[Similarity search](#toc0_)

In [8]:
query = "how many columns are?"

ans = vectorstore_unstructured_local.similarity_search(query)
print("-------Unstructured local Loader----------\n")
print(ans[0].page_content)


ans_2 = vectorstore_unstructured_api.similarity_search(query)
print("--------Unstructured api loader------------\n")
print(ans_2[0].page_content)


-------Unstructured local Loader----------

Column 1 Column 2 Row 1, Cell 1 Row 1, Cell 2 Row 2, Cell 1 Row 2, Cell 2
--------Unstructured api loader------------

Column 1 Column 2 Row 1, Cell 1 Row 1, Cell 2 Row 2, Cell 1 Row 2, Cell 2
